# Using Optimization Techniques to Generate Winning DraftKings Lineups
___
Until I build one from scratch I'm going to be testing out `pydfs-lineup-optimizer` to generate optimal lineups with projections that I already have. More information about `pydfs-lineup-optimizer` can be found [here](https://github.com/DimaKudosh/pydfs-lineup-optimizer/tree/master/pydfs_lineup_optimizer/sites/draftkings).

**Here is one quick example of generating lineups based on average points per game.**

In [27]:
import pandas as pd
from pydfs_lineup_optimizer import get_optimizer, Site, Sport, CSVLineupExporter

In [28]:
optimizer = get_optimizer(Site.DRAFTKINGS, Sport.FOOTBALL)

optimizer.load_players_from_csv('player-lists/Week-12/DKSalaries.csv')
for lineup in optimizer.optimize(n=5, max_exposure=0.3):
    print(lineup.players)
    print(lineup.fantasy_points_projection)
    print(lineup.salary_costs)

[Ben Roethlisberger QB (PIT), Albert Wilson WR (MIA), Emmanuel Sanders WR (DEN), Golden Tate WR (PHI), Melvin Gordon RB (LAC), Christian McCaffrey RB (CAR), Eric Ebron TE (IND), James Conner RB (PIT), Broncos  DST (DEN)]
170.76
50000.0
[Ben Roethlisberger QB (PIT), Albert Wilson WR (MIA), Emmanuel Sanders WR (DEN), Golden Tate WR (PHI), Melvin Gordon RB (LAC), Christian McCaffrey RB (CAR), Eric Ebron TE (IND), James Conner RB (PIT), Dolphins  DST (MIA)]
170.66
49800.0
[Ryan Fitzpatrick QB (TB), Antonio Brown WR (PIT), Tyler Boyd WR (CIN), DeSean Jackson WR (TB), Saquon Barkley RB (NYG), T.J. Yeldon RB (JAX), Zach Ertz TE (PHI), Marshawn Lynch RB (OAK), Dolphins  DST (MIA)]
160.66
49900.0
[Ryan Fitzpatrick QB (TB), Antonio Brown WR (PIT), Tyler Boyd WR (CIN), DeSean Jackson WR (TB), Saquon Barkley RB (NYG), T.J. Yeldon RB (JAX), Zach Ertz TE (PHI), Marshawn Lynch RB (OAK), Cardinals  DST (ARI)]
160.06
49900.0
[Cam Newton QB (CAR), Odell Beckham WR (NYG), T.Y. Hilton WR (IND), Mike Evans

In [29]:
problematic_names_dict = {'Melvin Gordon':'Melvin Gordon III', 'Odell Beckham':'Odell Beckham Jr.',\
                          'Will Fuller':'Will Fuller V', 'Marvin Jones':'Marvin Jones Jr.',\
                          'Willie Snead':'Willie Snead IV', 'Ted Ginn':'Ted Ginn Jr.', \
                          'Allen Robinson':'Allen Robinson II', 'Patrick Mahomes II':'Patrick Mahomes'}

## Now instead of just using the average points per game, we'll bring in our other projections that *should* be more accurate.

___
#### 1. Ballers

In [30]:
current_week = 13

qbs = pd.read_csv('projections/Week-{}/DFS - DraftKings QB.csv'.format(current_week))
rbs = pd.read_csv('projections/Week-{}/DFS - DraftKings RB.csv'.format(current_week))
wrs = pd.read_csv('projections/Week-{}/DFS - DraftKings WR.csv'.format(current_week))
tes = pd.read_csv('projections/Week-{}/DFS - DraftKings TE.csv'.format(current_week))
dst = pd.read_csv('projections/Week-{}/DFS - DraftKings DST.csv'.format(current_week))
full_team_names = list(dst.sort_values('Player')['Player'])

In [31]:
footballers = pd.concat([qbs,rbs,wrs,tes,dst])
footballers.rename(columns={"Player": "Name"}, inplace=True)
footballers.Name.replace(problematic_names_dict, inplace=True)

___
#### 2. Sportsline

In [32]:
sportsline = pd.read_csv('projections/Week-{}/projections.csv'.format(current_week))
sportsline.rename(columns={"PLAYER": "Name"}, inplace=True)
sportsline.Name.replace(problematic_names_dict, inplace=True)
abbv_team_names = list(sportsline.loc[sportsline.POS == 'DEF', :].sort_values('Name')['Name'])
defenses_dict = dict(zip(abbv_team_names, full_team_names))
sportsline['Name'].replace(defenses_dict, inplace = True)
sportsline.head(2)

,Name,POS,TEAM,OPP,FP,fd exp,dk exp,PPR,CONS%,FREB%,PASSYD,RUSHYD,RECYD
0,Patrick Mahomes,QB,KC,@OAK,35.4,24.6,28.4,35.4,60%,50%,379.0,18.0,NaN
1,Cam Newton,QB,CAR,@TB,28.1,22.5,24.7,28.1,49%,53%,300.0,42.0,NaN


___
#### 3. FantasyPros

In [33]:
fp_qb = pd.read_csv('projections/Week-{}/FantasyPros_Fantasy_Football_Projections_QB.csv'.format(current_week))
fp_rb = pd.read_csv('projections/Week-{}/FantasyPros_Fantasy_Football_Projections_RB (1).csv'.format(current_week))
fp_wr = pd.read_csv('projections/Week-{}/FantasyPros_Fantasy_Football_Projections_WR (1).csv'.format(current_week))
fp_te = pd.read_csv('projections/Week-{}/FantasyPros_Fantasy_Football_Projections_TE (1).csv'.format(current_week))
fp_dst = pd.read_csv('projections/Week-{}/FantasyPros_Fantasy_Football_Projections_DST.csv'.format(current_week))

In [34]:
for df in [fp_qb, fp_rb, fp_wr, fp_te, fp_dst]:
    df.drop([col for col in df.columns if col not in ['Player', 'FPTS']], axis=1, inplace=True)

In [35]:
fantasy_pros = pd.concat([fp_qb, fp_rb, fp_wr, fp_te, fp_dst])
fantasy_pros.rename(columns={"Player": "Name"}, inplace=True)
fantasy_pros.Name.replace(problematic_names_dict, inplace=True)

___
#### Averaging the projections:

In [36]:
total_projections = pd.merge(footballers, sportsline, on="Name")
total_projections = pd.merge(total_projections, fantasy_pros, on="Name")
total_projections['Salary'] = total_projections.Salary.apply(lambda x: int(x.replace(',', '')))
total_projections['Pts'] = (total_projections.Pts + total_projections.FP + total_projections.FPTS) / 3
total_projections['Pts Per 1k'] = total_projections['Pts'] / (total_projections.Salary / 1000)
total_projections.drop(labels=list(total_projections.columns[5:]), axis=1, inplace=True)
total_projections.Name.replace(dict(zip(full_team_names,[(i.split(' ')[-1] + ' ') for i in full_team_names])), inplace=True)
total_projections.head(3)

,Name,Opp,Pts,Salary,Pts Per 1k
0,Cam Newton,at TB,27.066667,6600,4.101010
1,Jared Goff,at DET,26.133333,6400,4.083333
2,Jameis Winston,vs. CAR,22.133333,6000,3.688889


___
#### Bringing in the player list from DraftKings:

In [37]:
# Make sure you're selecting the correct file:
player_list=pd.read_csv('player-lists/Week-13/DKSalaries (1).csv')
player_list.head(3)

,Position,Name + ID,Name,ID,Roster Position,Salary,Game Info,TeamAbbrev,AvgPointsPerGame
0,RB,Todd Gurley II (11704118),Todd Gurley II,11704118,RB/FLEX,9300,LAR@DET 12/02/2018 01:00PM ET,LAR,28.49
1,WR,Tyreek Hill (11704354),Tyreek Hill,11704354,WR/FLEX,9100,KC@OAK 12/02/2018 04:05PM ET,KC,24.34
2,RB,Christian McCaffrey (11704120),Christian McCaffrey,11704120,RB/FLEX,8800,CAR@TB 12/02/2018 01:00PM ET,CAR,25.32


> Merging those 2 and making the swap:

In [38]:
master_projections = pd.merge(left=player_list, right=total_projections, how='inner', on='Name', suffixes=['_DK', '_tp'])
master_projections['AvgPointsPerGame'] = master_projections['Pts Per 1k'] # Experiment with Pts OR Pts Per 1k
master_projections.drop(['Opp', 'Pts', 'Salary_tp', 'Pts Per 1k'],axis=1, inplace=True)
master_projections.rename(columns={'Salary_DK':'Salary'}, inplace=True)
master_projections.to_csv('projections/Week-{}/master_DK_projections.csv'.format(current_week))
master_projections.head(3)

,Position,Name + ID,Name,ID,Roster Position,Salary,Game Info,TeamAbbrev,AvgPointsPerGame
0,WR,Tyreek Hill (11704354),Tyreek Hill,11704354,WR/FLEX,9100,KC@OAK 12/02/2018 04:05PM ET,KC,2.641026
1,RB,Christian McCaffrey (11704120),Christian McCaffrey,11704120,RB/FLEX,8800,CAR@TB 12/02/2018 01:00PM ET,CAR,2.893939
2,WR,DeAndre Hopkins (11704356),DeAndre Hopkins,11704356,WR/FLEX,8200,CLE@HOU 12/02/2018 01:00PM ET,HOU,2.825203


___
# Alright! Now let's just use our optimizer tools from above to generate some better lineups

In [39]:
# out_players = list(master_projections.loc[(master_projections['Injury Indicator'] == 'IR') | (master_projections['Injury Indicator'] == 'O'), :]['Name'].values)
low_scorers = list(master_projections.loc[master_projections['AvgPointsPerGame'] < 0.80, :]['Name'].values)

all_removals = low_scorers
all_removals = list(set(all_removals))

In [40]:
optimizer = get_optimizer(Site.DRAFTKINGS, Sport.FOOTBALL)

optimizer.load_players_from_csv('projections/Week-{}/master_DK_projections.csv'.format(current_week))

# Removing certain players
for each_player in all_removals:
    optimizer.remove_player(each_player)

# Setting positional constraints
optimizer.set_players_with_same_position({'TE': 0})
# No stacking on this one

# Making more randomness
optimizer.set_deviation(0.06, 0.24)

# Setting a minimum salary cap
optimizer.set_min_salary_cap(48000)

# Generating 20 lineups
for lineup in optimizer.optimize(n=7, max_exposure=.35):
    print(lineup, '\n')

 1. QB    Cam Newton                    QB    CAR            4.101   6600.0$   
 2. WR1   Chris Godwin                  WR    TB             2.786   3900.0$   
 3. WR2   Brandin Cooks                 WR    LAR            2.729   7000.0$   
 4. WR3   DeAndre Hopkins               WR    HOU            2.825   8200.0$   
 5. RB1   Spencer Ware                  RB    KC             4.05    4000.0$   
 6. RB2   Aaron Jones                   RB    GB             3.209   6700.0$   
 7. TE    Eric Ebron                    TE    IND            3.024   4200.0$   
 8. FLEX  Phillip Lindsay               RB    DEN            3.179   5400.0$   
 9. DST   Rams                          DST   LAR            4.782   2600.0$   

Fantasy Points 30.685
Salary 48600.0 

 1. QB    Jared Goff                    QB    LAR            4.083   6400.0$   
 2. WR1   Chris Godwin                  WR    TB             2.786   3900.0$   
 3. WR2   Brandin Cooks                 WR    LAR            2.729   7000.0$   


In [57]:
exporter = CSVLineupExporter(optimizer.optimize(n=20, max_exposure=.25))
exporter.export('submissions/Week-{}/draftkings-lineups-nostacks.csv'.format(current_week))

In [58]:
optimizer = get_optimizer(Site.DRAFTKINGS, Sport.FOOTBALL)

optimizer.load_players_from_csv('projections/Week-{}/master_DK_projections.csv'.format(current_week))

# Removing certain players
for each_player in all_removals:
    optimizer.remove_player(each_player)

# Setting positional constraints
optimizer.set_players_with_same_position({'TE': 0})
optimizer.set_positions_for_same_team(['QB', 'WR'])

# Making more randomness
optimizer.set_deviation(0.06, 0.24)

# Setting a minimum salary cap
optimizer.set_min_salary_cap(48000)

# Generating 20 lineups
for lineup in optimizer.optimize(n=20, max_exposure=.25):
    print(lineup, '\n')

 1. QB    Cam Newton                    QB    CAR            4.054   6200.0$   
 2. WR1   Michael Thomas                WR    NO             2.765   8800.0$   
 3. WR2   Devin Funchess                WR    CAR            2.242   5100.0$   
 4. WR3   DeAndre Hopkins               WR    HOU            2.684   7900.0$   
 5. RB1   David Johnson                 RB    ARI            2.836   7500.0$   
 6. RB2   Alex Collins                  RB    BAL            2.697   4400.0$   
 7. TE    Ricky Seals-Jones             TE    ARI            2.77    2900.0$   
 8. FLEX  Evan Engram                   TE    NYG            2.732   4100.0$   
 9. DST   Cardinals                     DST   ARI            4.473   3100.0$   

Fantasy Points 27.252
Salary 50000.0 

 1. QB    Cam Newton                    QB    CAR            4.054   6200.0$   
 2. WR1   Michael Thomas                WR    NO             2.765   8800.0$   
 3. WR2   Devin Funchess                WR    CAR            2.242   5100.0$   


 1. QB    Carson Wentz                  QB    PHI            3.709   6300.0$   
 2. WR1   Kenny Golladay                WR    DET            2.362   5800.0$   
 3. WR2   Odell Beckham                 WR    NYG            2.468   8400.0$   
 4. WR3   Nelson Agholor                WR    PHI            2.065   4100.0$   
 5. RB1   Alvin Kamara                  RB    NO             2.785   8200.0$   
 6. RB2   Dion Lewis                    RB    TEN            2.625   4800.0$   
 7. TE    Greg Olsen                    TE    CAR            2.587   5000.0$   
 8. FLEX  Jared Cook                    TE    OAK            2.53    3900.0$   
 9. DST   Ravens                        DST   BAL            3.698   3200.0$   

Fantasy Points 24.828
Salary 49700.0 

 1. QB    Matt Ryan                     QB    ATL            3.754   6100.0$   
 2. WR1   Seth Roberts                  WR    OAK            2.211   3000.0$   
 3. WR2   Julio Jones                   WR    ATL            2.227   8500.0$   


In [59]:
exporter = CSVLineupExporter(optimizer.optimize(n=20, max_exposure=.25))
exporter.export('submissions/Week-{}/draftkings-lineups-qbwr.csv'.format(current_week))

In [60]:
# Doing the second half with a RB + D stack

optimizer = get_optimizer(Site.DRAFTKINGS, Sport.FOOTBALL)

optimizer.load_players_from_csv('projections/Week-{}/master_DK_projections.csv'.format(current_week))

# Removing certain players
for each_player in all_removals:
    optimizer.remove_player(each_player)

# Setting positional constraints
optimizer.set_players_with_same_position({'TE': 0})
optimizer.set_positions_for_same_team(['RB', 'DST'])

    
# Making more randomness
optimizer.set_deviation(0.06, 0.24)

# Setting a minimum salary cap
optimizer.set_min_salary_cap(48000)

# Generating 20 lineups
for lineup in optimizer.optimize(n=20, max_exposure=.25):
    print(lineup, '\n')

 1. QB    Cam Newton                    QB    CAR            4.054   6200.0$   
 2. WR1   Michael Thomas                WR    NO             2.765   8800.0$   
 3. WR2   Brandon LaFell                WR    OAK            2.314   3400.0$   
 4. WR3   DeAndre Hopkins               WR    HOU            2.684   7900.0$   
 5. RB1   David Johnson                 RB    ARI            2.836   7500.0$   
 6. RB2   Mark Ingram                   RB    NO             2.801   4700.0$   
 7. TE    Ricky Seals-Jones             TE    ARI            2.77    2900.0$   
 8. FLEX  Evan Engram                   TE    NYG            2.732   4100.0$   
 9. DST   Cardinals                     DST   ARI            4.473   3100.0$   

Fantasy Points 27.428
Salary 48600.0 

 1. QB    Cam Newton                    QB    CAR            4.054   6200.0$   
 2. WR1   Michael Thomas                WR    NO             2.765   8800.0$   
 3. WR2   Brandon LaFell                WR    OAK            2.314   3400.0$   


 1. QB    Matt Ryan                     QB    ATL            3.754   6100.0$   
 2. WR1   Seth Roberts                  WR    OAK            2.211   3000.0$   
 3. WR2   Keenan Allen                  WR    LAC            2.363   6700.0$   
 4. WR3   Amari Cooper                  WR    DAL            2.352   5400.0$   
 5. RB1   Saquon Barkley                RB    NYG            2.854   8700.0$   
 6. RB2   Tevin Coleman                 RB    ATL            2.711   5300.0$   
 7. TE    Jared Cook                    TE    OAK            2.53    3900.0$   
 8. FLEX  Christian McCaffrey           RB    CAR            2.692   8000.0$   
 9. DST   Panthers                      DST   CAR            3.536   2800.0$   

Fantasy Points 25.003
Salary 49900.0 

 1. QB    Dak Prescott                  QB    DAL            3.699   5200.0$   
 2. WR1   Seth Roberts                  WR    OAK            2.211   3000.0$   
 3. WR2   Devin Funchess                WR    CAR            2.242   5100.0$   


In [61]:
exporter = CSVLineupExporter(optimizer.optimize(n=20, max_exposure=.25))
exporter.export('submissions/Week-{}/draftkings-lineups-rbdst.csv'.format(current_week))

In [62]:
# QB + TE stack

optimizer = get_optimizer(Site.DRAFTKINGS, Sport.FOOTBALL)

optimizer.load_players_from_csv('projections/Week-{}/master_DK_projections.csv'.format(current_week))

# Removing certain players
for each_player in all_removals:
    optimizer.remove_player(each_player)

# Setting positional constraints
optimizer.set_players_with_same_position({'TE': 0})
optimizer.set_positions_for_same_team(['QB', 'TE'])

# Making more randomness
optimizer.set_deviation(0.06, 0.24)

# Setting a minimum salary cap
optimizer.set_min_salary_cap(47000)

# Generating 20 lineups
for lineup in optimizer.optimize(n=20, max_exposure=.25):
    print(lineup, '\n')

 1. QB    Cam Newton                    QB    CAR            4.054   6200.0$   
 2. WR1   Michael Thomas                WR    NO             2.765   8800.0$   
 3. WR2   Brandon LaFell                WR    OAK            2.314   3400.0$   
 4. WR3   DeAndre Hopkins               WR    HOU            2.684   7900.0$   
 5. RB1   David Johnson                 RB    ARI            2.836   7500.0$   
 6. RB2   Mark Ingram                   RB    NO             2.801   4700.0$   
 7. TE    Ricky Seals-Jones             TE    ARI            2.77    2900.0$   
 8. FLEX  Greg Olsen                    TE    CAR            2.587   5000.0$   
 9. DST   Cardinals                     DST   ARI            4.473   3100.0$   

Fantasy Points 27.283
Salary 49500.0 

 1. QB    Cam Newton                    QB    CAR            4.054   6200.0$   
 2. WR1   Michael Thomas                WR    NO             2.765   8800.0$   
 3. WR2   Kenny Golladay                WR    DET            2.362   5800.0$   


 1. QB    Matt Ryan                     QB    ATL            3.754   6100.0$   
 2. WR1   Seth Roberts                  WR    OAK            2.211   3000.0$   
 3. WR2   Emmanuel Sanders              WR    DEN            2.317   6200.0$   
 4. WR3   Sterling Shepard              WR    NYG            2.233   5300.0$   
 5. RB1   Saquon Barkley                RB    NYG            2.854   8700.0$   
 6. RB2   Alvin Kamara                  RB    NO             2.785   8200.0$   
 7. TE    Austin Hooper                 TE    ATL            2.281   4500.0$   
 8. FLEX  Dion Lewis                    RB    TEN            2.625   4800.0$   
 9. DST   Ravens                        DST   BAL            3.698   3200.0$   

Fantasy Points 24.758
Salary 50000.0 

 1. QB    Carson Wentz                  QB    PHI            3.709   6300.0$   
 2. WR1   Seth Roberts                  WR    OAK            2.211   3000.0$   
 3. WR2   Keenan Allen                  WR    LAC            2.363   6700.0$   


In [63]:
exporter = CSVLineupExporter(optimizer.optimize(n=20, max_exposure=.2))
exporter.export('submissions/Week-{}/draftkings-lineups-qbte.csv'.format(current_week))